In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Alopecia/GSE66664'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome analysis reveals differences in vasculature signalling between human dermal papilla cells from balding and non-balding scalp"
!Series_summary	"Transcriptome analysis of hTERT-immortalized balding (BAB) and non-balding (BAN) dermal papilla cells derived from frontal and occipital scalp of male patients with androgenetic alopecia Hamilton grade IV. Interrogation of transcriptome differences between BAB and BAN after dihydrotestosterone (DHT, active metabolite of androgen) treatment revealed significant enrichment of vasculature-related genes among down-regulated genes in BAB compared to BAN."
!Series_overall_design	"RNA obtained from BAB and BAN after treatment with 1nM or 10nM of DHT, 2-3 replicates for each condition"
Sample Characteristics Dictionary:
{0: ['cell line: BAB', 'cell line: BAN'], 1: ['agent: DHT'], 2: ['dose: 10nM', 'dose: 1nM'], 3: ['time (treatment duration): 0h', 'time (treatment duration): 12h', 'time (treatment dur

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # Because the background information mentions Transcriptome analysis

# Data Availability and Data Type Conversion
# After reviewing the sample characteristics dictionary, it appears that:
# - 'Alopecia' can be inferred from 'cell line: BAB' and 'cell line: BAN', so it's available in key 0
# - Age: Not explicitly mentioned, assume not available
# - Gender: Not explicitly mentioned but study is about male patients only, so it's not available

trait_row = 0

# Converting functions
def convert_trait(value):
    value_split = value.split(':')[-1].strip().lower()
    if 'bab' in value_split:
        return 1
    elif 'ban' in value_split:
        return 0
    else:
        return None

# Since age and gender data are not available, their rows and conversion functions remain None
convert_age = convert_gender = None

# Save cohort information
save_cohort_info('GSE66664', './preprocessed/Alopecia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Alopecia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Alopecia/trait_data/GSE66664.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1627302': [1], 'GSM1627303': [1], 'GSM1627304': [1], 'GSM1627305': [1], 'GSM1627306': [1], 'GSM1627307': [1], 'GSM1627308': [1], 'GSM1627309': [1], 'GSM1627310': [1], 'GSM1627311': [1], 'GSM1627312': [1], 'GSM1627313': [1], 'GSM1627314': [1], 'GSM1627315': [1], 'GSM1627316': [1], 'GSM1627317': [1], 'GSM1627318': [1], 'GSM1627319': [1], 'GSM1627320': [1], 'GSM1627321': [1], 'GSM1627322': [1], 'GSM1627323': [1], 'GSM1627324': [1], 'GSM1627325': [1], 'GSM1627326': [1], 'GSM1627327': [1], 'GSM1627328': [1], 'GSM1627329': [1], 'GSM1627330': [1], 'GSM1627331': [1], 'GSM1627332': [1], 'GSM1627333': [1], 'GSM1627334': [1], 'GSM1627335': [1], 'GSM1627336': [1], 'GSM1627337': [1], 'GSM1627338': [1], 'GSM1627339': [1], 'GSM1627340': [1], 'GSM1627341': [1], 'GSM1627342': [1], 'GSM1627343': [1], 'GSM1627344': [1], 'GSM1627345': [1], 'GSM1627346': [1], 'GSM1627347': [1], 'GSM1627348': [1], 'GSM1627349': [1], 'GSM1627350': [1], 'GSM1627351': [1], 'GSM1627352': [1], 'GSM1627353': [1], 'GSM1627354

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Alopecia/gene_data/GSE66664.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Alopecia')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE66664', './preprocessed/Alopecia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Alopecia/GSE66664.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Alopecia', the least common label is '0.0' with 69 occurrences. This represents 49.29% of the dataset.
The distribution of the feature 'Alopecia' in this dataset is fine.

